In [11]:
import numpy as np
from typing import List, Tuple


def get_element(arr: np.ndarray, i: int, j: int):
    if j <= 0 or i <= 0 or i >= arr.shape[0] or j >= arr.shape[1]:
        return -1
    return arr[i, j]


def hough_lines(
    image: np.ndarray,
    rho: float,
    theta: float,
    threshold: int,
    min_theta: float = 0,
    max_theta: float = np.pi,
) -> list[tuple[float, float, int]]:
    """Finds lines in a binary image using the standard Hough transform.

    Arguments:
        image: 2d np.ndarray of bool
        rho: distance resolution
        theta: angle resolution
        theshold: only lines with votes > `threshold` are returned
        min_theta: minimal angle value
        max_theta: maximal angle value

    Returns:
        lines: a list of tuples (distance, angle, votes), detected lines sorted by votes in descending order
    """
    theta_ = np.arange(min_theta, max_theta, theta)
    rs_ = int((image.shape[0] + image.shape[1]) / rho)
    rs = np.arange(-rs_, rs_ + rho, rho)
    accumulator = np.zeros((len(theta_), 2 * rs_), dtype=int)
    cos_theta = np.cos(theta_)
    sin_theta = np.sin(theta_)
    theta_idx = np.arange(len(theta_))

    for y, x in np.argwhere(image):
        r = (np.round(x * cos_theta + y * sin_theta) + rs_).astype(int)
        accumulator[theta_idx, r] += 1

    # First bug
    result = []
    for i, j in np.argwhere(accumulator >= threshold):
        if (
            get_element(accumulator, i - 1, j) < accumulator[i, j]
            and get_element(accumulator, i + 1, j) <= accumulator[i, j]
            and get_element(accumulator, i, j - 1) < accumulator[i, j]
            and get_element(accumulator, i, j + 1) <= accumulator[i, j]
        ):
            try:
                temp = rs[j]
            except:
                return [(255, 123, 9)]

            result.append((temp, theta_[i], accumulator[i, j]))
    # First bug

    try:
        lines = np.array(result)
        lines = lines[np.argsort(lines[:, 2])[::-1]]
    except:
        return [(255.0, 123.0, 599)]

    return [
        (x, y, z) for x, y, z in zip(lines[:, 0], lines[:, 1], lines[:, 2].astype(int))
    ]

In [12]:
import cv2

image = cv2.Canny(cv2.imread("lines.jpg"), 500, 550).astype(bool)
lines = hough_lines(image, 1, np.pi / 360, 210)


In [13]:
lines

[(2739.0, 0.3141592653589793, 636),
 (2749.0, 0.3141592653589793, 580),
 (2505.0, 0.7853981633974483, 580),
 (1381.0, 2.6354471705114375, 429),
 (2811.0, 2.0943951023931953, 420),
 (3222.0, 1.3700834628155487, 393),
 (3233.0, 1.3700834628155487, 373),
 (2821.0, 2.0943951023931953, 369),
 (2879.0, 2.199114857512855, 328),
 (2516.0, 0.7853981633974483, 305),
 (1392.0, 2.6354471705114375, 288),
 (2174.0, 2.91469985083053, 286),
 (3753.0, 0.6457718232379019, 283),
 (2346.0, 1.8151424220741028, 281),
 (2871.0, 2.199114857512855, 279),
 (2354.0, 1.8151424220741028, 271),
 (2662.0, 1.2740903539558606, 265),
 (2344.0, 1.8151424220741028, 263),
 (2670.0, 1.2740903539558606, 259),
 (2352.0, 1.8151424220741028, 258),
 (2183.0, 2.91469985083053, 255),
 (1456.0, 2.8012534494508987, 225),
 (1463.0, 2.792526803190927, 218),
 (1465.0, 2.8012534494508987, 218),
 (1472.0, 2.792526803190927, 217),
 (3764.0, 0.6457718232379019, 212)]